In [2]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=False)
import numpy as np
import matplotlib.pyplot as plt
!pip install -U tifffile[all]
from tifffile import tifffile as tiff
!pip install pip install readlif
from readlif.reader import LifFile
!pip install diplib==3.4.0
import diplib as dip

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/25.8 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 0.6/25.8 MB 10.2 MB/s eta 0:00:03
   - -------------------------------------- 1.0/25.8 MB 11.0 MB/s eta 0:00:03
   --- ------------------------------------ 2.1/25.8 MB 13.5 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/25.8 MB 17.4 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/25.8 MB 14.2 MB/s eta 0:00:02
   ----- ---------------------------------- 3.4/25.8 MB 12.1 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/25.8 MB 10.3 MB/s eta 0:00:03
   ----- ---------------------------------- 3.8/25.8 MB 9.6 MB/s eta 0:00:03
   ------- -------------------------------- 4.6/25.8 MB 10.5 MB/s eta 0:00:03
   ---------- ----------------------------- 6.9/25

# Read and unpack lif-file, and convert to numpy. Then Store as 3D tiff.
You need to unpack your lif file and convert it to a 3D tiff-file. The two functions below can be used for reading lif->numpy and storing numpy->3D-tiff.

In [111]:
def Lif_to_Numpy ( image_stack ) :
  Channels = [ list () for _ in range ( image_stack.channels ) ]
  for C in range ( image_stack.channels ) :
    for frame in image.get_iter_z ( c = C ) :
      Channels [C].append (np.array ( frame ) )
  return np.array ( Channels )
#*******************************************************************************
def Save_Stacked_Image ( image , filename , path = '') :
  image = np.array ( dip.Image ( image ) )
  tiff.imwrite(path+filename+'.tiff', image)
#*******************************************************************************

def enhance_signal(image):
    channel, depth, y_size, x_size = image.shape
    print(image.shape)

    enhanced_image = np.zeros_like(image)

    for i in range(channel):
        for j in range(depth):
            img_slice = dip.Image(image[i, j, :, :])
            
            # plt.imshow(img_slice)
            # plt.show()
            better = dip.Sharpen(img_slice)
            mask = dip.OtsuThreshold(better)
            better = better * mask
            # plt.imshow(better)
            # plt.show()
            # break
            enhanced_image[i, j, :, :] = better

    return enhanced_image
      
lif_file = LifFile ("C:/Users/bramv/OneDrive - Square Four/Documenten/Leiden/23-24/IA/A6/Images/team08.lif" ) # XXX Fill in your filename and path
for idx , image in enumerate ( lif_file.get_iter_image () ) :
  if idx == 0:
    img = Lif_to_Numpy ( image )
    enh_img = enhance_signal(img)
  else:
    break
  Save_Stacked_Image ( enh_img , f"C:/Users/bramv/OneDrive - Square Four/Documenten/Leiden/23-24/IA/A6/Images/Image_for_Huygens_{idx}")

(3, 32, 512, 512)


# Reading 3D tiff stacks
If you have already converted your lif files to tiff you can read it as follows.

In [ ]:
def ReadStackedTiff ( filename , path = '/content/drive/MyDrive/') : # XXX Fill in your path
  image_stack = tiff.imread ( path + filename )
  print(f"The shape of { filename } is { image_stack.shape }")
  return image_stack
#*******************************************************************************
def Stack_to_dip_list ( imageStack ) :
  return [ dip.Image ( image ) for image in imageStack ]
#*******************************************************************************
image_stack = ReadStackedTiff ( "Image_for_Huygens_0.tiff" )
dipImages_stack = Stack_to_dip_list ( image_stack )

The shape of Image_for_Huygens_0.tiff is (31, 512, 512, 3)
